In [1]:
""" 
Preproccess EEG data from BrainVision
Saves in .mat format, segmented by triggers
"""

################
## Imports
################
import mne
import os
import glob
import librosa
import numpy as np
import pandas as pd
from scipy.io import savemat
from scipy.signal import hilbert, resample, correlate
from scipy.signal import butter, filtfilt
import matplotlib.pyplot as plt
import naplib as nl

from IPython.display import Audio, display

In [2]:
def lowpass_filter(data, cutoff, fs, order=4):
    b, a = butter(order, cutoff / (0.5 * fs), btype='low')
    return filtfilt(b, a, data)

In [3]:
stim_i = 3
stimuli_list = ['../../Stimuli/Cindy/speech_only_long_22kHz/jane_eyre_05_part1.wav',
               '../../Stimuli/Cindy/piano_only_long_22kHz/piano_4_1_22050Hz.wav',
               '../../Stimuli/Cindy/speech_only_long_22kHz/jane_eyre_05_part2.wav',
               '../../Stimuli/Cindy/piano_only_long_22kHz/piano_4_2_22050Hz.wav']
# stimuli_sr = 22050

In [4]:
################
## EDIT THIS PART
################

# file = '../EEG_data_test/trig_test_2/Untitled2.vhdr'
file = f'../../Data/Samet/single{stim_i+1}.vhdr'

filename = file.split('/')[-1].split('.')[0]
exp_type = file.split('/')[-1].split('.')[0].split('_')[-1]
# exp_type = 'mixed' 

output_dir = '../../Data/Samet/Preprocessed/preprocessed_single_01_15Hz'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

overwrite = False

In [5]:
################
## Parameters
################
plot = False
# FS_ORIG = 25000  # Hz

# Preprocessing
# Notch filtering
notch_applied = False
freq_notch = 60

# Bandpass filtering
bpf_applied = True
freq_low   = 0.1
freq_high  = 15
bandpass = str(freq_low) + '-' + str(freq_high)
ftype = 'butter'
order = 3

# Spherical interpolation
int_applied = False
interpolation = 'spline'

# Rereferencing using average of mastoids electrodes
reref_applied = True
reref_type = 'average'  #channels or average
reref_channels = None

# Downsampling
down_applied = True
downfreq = 128
if not down_applied:
    downfreq = 'N/A'

In [6]:
################
## Read and crop data
################
raw = mne.io.read_raw_brainvision(file, preload=True)

#get info from raw
FS_ORIG = raw.info['sfreq']
ch_names = raw.ch_names
events, event_id = mne.events_from_annotations(raw)

#crop to start
trial_start = events[0][0] / FS_ORIG
trial_end = raw.times[-1]
eeg = raw.copy().crop(tmin = trial_start,
                      tmax = trial_end)

Extracting parameters from ../../Data/Samet/single4.vhdr...
Setting channel info structure...
Reading 0 ... 561779  =      0.000 ...   561.779 secs...


/var/folders/p8/sfp3f3z124xdkgdf3ksb_kp00000gn/T/ipykernel_5422/1919079818.py:4: RuntimeWarning: No coordinate information found for channels ['Soundwave']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  raw = mne.io.read_raw_brainvision(file, preload=True)
/var/folders/p8/sfp3f3z124xdkgdf3ksb_kp00000gn/T/ipykernel_5422/1919079818.py:4: RuntimeWarning: Not setting position of 1 misc channel found in montage:
['Soundwave']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  raw = mne.io.read_raw_brainvision(file, preload=True)


Used Annotations descriptions: ['New Segment/', 'Stimulus/S  3']


In [7]:
# Load long audio at 1000 Hz
long_audio = eeg.get_data()[31,:]
long_sr = 1000

# Load short audio at its native rate (22050 Hz)
stimname = stimuli_list[stim_i].split('/')[-1][:-4]
if stim_i in [0,2]:
    short_audio, short_sr = librosa.load(f'../../Stimuli/Cindy/speech_only_long_22kHz/{stimname}.wav', sr=None)
elif stim_i in [1,3]:
    short_audio, short_sr = librosa.load(f'../../Stimuli/Cindy/piano_only_long_22kHz/{stimname}.wav', sr=None)
    

# Resample short audio to match long audio's sampling rate
short_audio_resampled = librosa.resample(short_audio, orig_sr=short_sr, target_sr=long_sr)

# Normalize both signals
long_audio = (long_audio - np.mean(long_audio)) / np.std(long_audio)
short_audio_resampled = (short_audio_resampled - np.mean(short_audio_resampled)) / np.std(short_audio_resampled)

# Cross-correlation to find best match
correlation = correlate(long_audio, short_audio_resampled, mode='valid')
best_match_index = np.argmax(correlation)
end_index = best_match_index + len(short_audio_resampled)

print(f"Best match found at index range: {best_match_index} to {end_index}")
print(events[1,0])

Best match found at index range: 5132 to 556467
4973


In [8]:
################
## SEGMENT DATA
################

eeg = raw.copy().crop(tmin=best_match_index / FS_ORIG, tmax = end_index / FS_ORIG)

################
## Preprocess
################
df_pre = pd.DataFrame()

## -------------
## Select channels
## -------------
eeg_channels = ch_names[0:31]
eeg = eeg.pick_channels(eeg_channels)
if plot:
    eeg.plot(start=100, duration=10, n_channels=len(raw.ch_names))

## -------------
## Notch filtering
## -------------
df_pre['notch_applied'] = [notch_applied]
if notch_applied:
    eeg = eeg.notch_filter(freqs=freq_notch)
    df_pre['notch'] = [freq_notch]
    if plot:
        eeg.plot()

## -------------
## BPFiltering
## -------------
df_pre['bpf_applied'] = [bpf_applied]
if bpf_applied:
    iir_params = dict(order=order, ftype=ftype)
    filter_params = mne.filter.create_filter(eeg.get_data(), eeg.info['sfreq'], 
                                            l_freq=freq_low, h_freq=freq_high, 
                                            method='iir', iir_params=iir_params)

    if plot:
        flim = (1., eeg.info['sfreq'] / 2.)  # frequencies
        dlim = (-0.001, 0.001)  # delays
        kwargs = dict(flim=flim, dlim=dlim)
        mne.viz.plot_filter(filter_params, eeg.info['sfreq'], compensate=True, **kwargs)
        # plt.savefig(os.path.join(output_dir, 'bpf_ffilt_shape.png'))

    eeg = eeg.filter(l_freq=freq_low, h_freq=freq_high, method='iir', iir_params=iir_params)
    df_pre['bandpass'] = [iir_params]
    df_pre['HPF'] = [freq_low]
    df_pre['LPF'] = [freq_high]
    if plot:
        eeg.plot()

## -------------
## Intrpolation
## -------------
df_pre['int_applied'] = [int_applied]
if int_applied: 
    eeg = eeg.interpolate_bads(reset_bads=False)  #, method=interpolation

    # Get the indices and names of the interpolated channels
    interp_inds = eeg.info['bads']
    interp_names = [eeg.info['ch_names'][i] for i in interp_inds]

    # Print the number and names of the interpolated channels
    print(f'{len(interp_inds)} channels interpolated: {interp_names}')

    df_pre['interpolation'] = [interpolation]
    df_pre['interp_inds'] = [interp_inds]
    df_pre['interp_names'] = [interp_names]

    if plot:
        eeg.plot()

## -------------
## Rereferencing
## -------------
df_pre['reref_applied'] = [reref_applied]
if reref_applied:
    if reref_type == 'average':
        # reref to average
        eeg = eeg.set_eeg_reference(ref_channels='average')
        df_pre['reref_type'] = [reref_type]
        df_pre['reref_channels'] = ['average']
        if plot:
            eeg.plot()

    elif reref_type == 'channels':
        # reref to a channel
        eeg = eeg.set_eeg_reference(ref_channels=reref_channels)
        df_pre['reref_type'] = [reref_type]
        df_pre['reref_channels'] = [reref_channels]
        if plot:
            eeg.plot()

## -------------
## Resampling
## -------------
df_pre['down_applied'] = [down_applied]
df_pre['downfreq'] = [downfreq]
if down_applied:
    eeg = eeg.resample(sfreq=downfreq)
    print(eeg.info)
    if plot:
        eeg.plot()

## -------------
## Stimuli
## -------------
curr_audio, sr = short_audio, short_sr
analytic_signal = hilbert(curr_audio)
envelope = np.abs(analytic_signal)

cutoff_freq = 8  # Hz
envelope = lowpass_filter(envelope, cutoff=cutoff_freq, fs=short_sr)

# _, envelope, _ = nl.preprocessing.filter_hilbert(curr_audio, stimuli_sr)
# envelope = np.squeeze(envelope)

duration_sec = len(envelope) / short_sr
n_target_samples = int(duration_sec * downfreq)

envelope = resample(envelope, n_target_samples)

## -------------
## Save preprocessing stages
## -------------
df_pre.to_csv(os.path.join(output_dir, filename+'_pp_record.csv'), index=False)


################
## Save files
################

eeg_tosave = eeg.get_data()
savename = filename + '_' + stimname +'.mat'
print(savename)
savemat(os.path.join(output_dir, savename), 
        #{'eeg_data': eeg_tosave[0:31]}
        {'eeg_data': eeg_tosave, 'stimuli': short_audio, 'envelope': envelope}
        )

#segment single to 10 mins
#else: 


NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Setting up band-pass filter from 0.1 - 15 Hz

IIR filter parameters
---------------------
Butterworth bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 12 (effective, after forward-backward)
- Cutoffs at 0.10, 15.00 Hz: -6.02, -6.02 dB

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.1 - 15 Hz

IIR filter parameters
---------------------
Butterworth bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 12 (effective, after forward-backward)
- Cutoffs at 0.10, 15.00 Hz: -6.02, -6.02 dB

EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
<Info | 9 non-empty values
 bads: []
 ch_names: Fp1, Fz, F3, F7, FT9, FC5, FC1, C3, T7, TP9, CP5, CP1, Pz, P3, ...
 chs: 31 EEG
 custom_ref_applied: True
 dig: 34 items (3 Cardinal, 31 EEG)
 highpass: 0.1 Hz
 lowpass: 15.0